## Galaxy-Galaxy Lenses with Power-Law - Full Sky Area 20,000 sq.deg
Author: Paras Sharma

We generate galaxy galaxy lenses population here.

In [1]:
# Standard imports
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

# Astropy imports
from astropy.io import fits
from astropy import units as u
from astropy.table import Table, vstack
from astropy.units import Quantity
from astropy import constants as const
import corner

# SLSim imports
import slsim.Pipelines as pipelines
import slsim.Sources as sources
import slsim.Deflectors as deflectors
from slsim.Lenses.lens_pop import LensPop
from slsim.Plots.lens_plots import LensingPlots
from slsim.Lenses.lens import Lens
from slsim.LOS.los_pop import LOSPop

# Lenstronomy, HierArc imports
from lenstronomy.LensModel.lens_model import LensModel
from hierarc.Sampling.ParamManager.cosmo_param import CosmoParam

from warnings import filterwarnings
filterwarnings("ignore")

**WARNING**: LOCAL JWST PRD VERSION PRDOPSSOC-068 DOESN'T MATCH THE CURRENT ONLINE VERSION PRDOPSSOC-071
Please consider updating pysiaf, e.g. pip install --upgrade pysiaf or conda update pysiaf
/home/paras/sharmapyenv/lib64/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/paras/repos/self/slsim/slsim/Plots/lens_plots.py:5: UserWarning: If you want to simulate images with Roman filters, please install the galsim module.
Note that this module is not supported on Windows
  from slsim.ImageSimulation.roman_image_simulation import simulate_roman_image


In [2]:
PROJECT_DIR = "~/projects/PDSPL"

## Generate SLSim Catalog

In [ ]:
# define a cosmology
cosmology = "FwCDM"  # Flat wCDM cosmology
# other options are: "FLCDM FwCDM", "w0waCDM", "oLCDM"
kwargs_cosmo_true = {"h0": 70, "om": 0.3, "w": -1}  # cosmological model of the forecast

# create astropy.cosmology instance of input cosmology
cosmo_param = CosmoParam(cosmology=cosmology)
cosmo_true = cosmo_param.cosmo(kwargs_cosmo_true)

# define a sky area
# sky_area_galaxies = Quantity(value=100, unit="deg2")

# load red and blue galaxy catalogs
red_galaxy_catalog = Table.read(f"{PROJECT_DIR}/data/skypy_galaxy_catalogs/SKYPY_RED_GALAXIES_{1000.0}_SQDEG_0.fits",
                                  format="fits")
blue_galaxy_catalog = Table.read(f"{PROJECT_DIR}/data/skypy_galaxy_catalogs/SKYPY_BLUE_GALAXIES_{100.0}_SQDEG_0.fits",
                                   format="fits") 


In [4]:
print("# of red galaxies:", len(red_galaxy_catalog))
print("# of blue galaxies:", len(blue_galaxy_catalog))

# of red galaxies: 84567650
# of blue galaxies: 131301420


In [ ]:
%%writefile ~/projects/PDSPL/notebooks/worker.py
# This entire cell will be saved as 'worker.py'

import numpy as np
import astropy.units as u
import astropy.constants as const
from astropy.cosmology import FlatLambdaCDM
from lenstronomy.LensModel.lens_model import LensModel

cosmo_true = FlatLambdaCDM(H0=70, Om0=0.3)

# --- THE WORKER FUNCTION ---
def extract_lens_properties(args):
    """
    Worker function that takes a tuple (id, lens_object), extracts all
    properties, and returns them as a single dictionary.
    """
    lens_id, lens = args
    
    # try:
    deflector = lens.deflector
    source = lens.source(index=0)

    z_D = lens.deflector_redshift
    z_S = lens.source_redshift_list[0]
    theta_E = lens.einstein_radius[0]
    sigma_v_D = deflector.velocity_dispersion()
    e1_mass_D, e2_mass_D = deflector.mass_ellipticity
    
    half_light_radii_arcsec = deflector.angular_size_light
    
    lenstronomy_kwargs = lens.lenstronomy_kwargs()
    lens_model_lenstronomy = LensModel(lens_model_list=lenstronomy_kwargs[0]["lens_model_list"])
    lenstronomy_kwargs_lens = lenstronomy_kwargs[1]["kwargs_lens"]
    
    deflector_center = deflector.deflector_center
    grid = np.linspace(-half_light_radii_arcsec, half_light_radii_arcsec, 500)
    xs, ys = np.meshgrid(grid + deflector_center[0], grid + deflector_center[1])
    
    kappa_map = lens_model_lenstronomy.kappa(xs, ys, kwargs=lenstronomy_kwargs_lens)
    mask = np.sqrt((xs - deflector_center[0])**2 + (ys - deflector_center[1])**2) < half_light_radii_arcsec / 2
    kappa_within_half_light_radii = np.nanmean(kappa_map[mask])

    D_s = cosmo_true.angular_diameter_distance(lens.source_redshift_list[0])
    D_d = cosmo_true.angular_diameter_distance(lens.deflector_redshift)
    D_ds = cosmo_true.angular_diameter_distance_z1z2(lens.deflector_redshift, lens.source_redshift_list[0])
    
    sigma_crit = (const.c**2 / (4 * np.pi * const.G)) * (D_s / (D_d * D_ds))
    sigma_crit = sigma_crit.to(u.Msun / u.pc**2).value
    surface_density = sigma_crit * kappa_within_half_light_radii

    surface_brightness_map = deflector.surface_brightness(xs, ys, band="g")
    mask_sb = np.sqrt((xs - deflector_center[0])**2 + (ys - deflector_center[1])**2) < half_light_radii_arcsec
    mean_surface_brightness = np.nanmean(surface_brightness_map[mask_sb])

    R_e_kpc_val = (cosmo_true.kpc_proper_per_arcmin(lens.deflector_redshift) * \
                    ((half_light_radii_arcsec * u.arcsec).to(u.arcmin))).to(u.kpc).value

    ### contrast ratio of images in mag difference
    contrast_ratio_i = lens.contrast_ratio(band="i", source_index = 0)
    contrast_ratio_r = lens.contrast_ratio(band="r", source_index = 0)
    contrast_ratio_g = lens.contrast_ratio(band="g", source_index = 0)
    contrast_ratio_z = lens.contrast_ratio(band="z", source_index = 0)
    contrast_ratio_y = lens.contrast_ratio(band="y", source_index = 0)

    # make all to have len of 4
    contrast_ratio_i = np.array(list(contrast_ratio_i) + [np.nan] * (4 - len(contrast_ratio_i)))
    contrast_ratio_r = np.array(list(contrast_ratio_r) + [np.nan] * (4 - len(contrast_ratio_r)))
    contrast_ratio_g = np.array(list(contrast_ratio_g) + [np.nan] * (4 - len(contrast_ratio_g)))
    contrast_ratio_z = np.array(list(contrast_ratio_z) + [np.nan] * (4 - len(contrast_ratio_z)))
    contrast_ratio_y = np.array(list(contrast_ratio_y) + [np.nan] * (4 - len(contrast_ratio_y)))

    ## magnifications of point images and extended images
    magnification_point = lens.point_source_magnification()[0]
    # convert to array of length 4
    magnification_point = np.array(list(magnification_point) + [np.nan] * (4 - len(magnification_point)))

    magnification_extended = lens.extended_source_magnification[0] # not an array, just a number

    return {
        "lens_id": lens_id, "z_D": z_D, "z_S": z_S, "theta_E": theta_E, "sigma_v_D": sigma_v_D,
        "stellar_mass_D": deflector.stellar_mass, "mag_S_i": source.extended_source_magnitude("i"),
        "mag_S_r": source.extended_source_magnitude("r"), "mag_S_g": source.extended_source_magnitude("g"),
        "mag_S_z": source.extended_source_magnitude("z"), "mag_S_y": source.extended_source_magnitude("y"),
        "mag_D_i": deflector.magnitude("i"), "mag_D_r": deflector.magnitude("r"), "mag_D_g": deflector.magnitude("g"),
        "mag_D_z": deflector.magnitude("z"), "mag_D_y": deflector.magnitude("y"), "size_D": deflector.angular_size_light,
        "e1_mass_D": e1_mass_D, "e2_mass_D": e2_mass_D, "e_mass_D": np.sqrt(e1_mass_D**2 + e2_mass_D**2),
        "gamma_pl": deflector.halo_properties['gamma_pl'], "R_e_kpc": R_e_kpc_val, "R_e_arcsec": half_light_radii_arcsec,
        "Sigma_half_Msun/pc2": surface_density, "surf_bri_mag/arcsec2": mean_surface_brightness,
        "num_images": lens.image_number[0],
        "contrast_ratio_i": contrast_ratio_i,
        "contrast_ratio_r": contrast_ratio_r,
        "contrast_ratio_g": contrast_ratio_g,
        "contrast_ratio_z": contrast_ratio_z,
        "contrast_ratio_y": contrast_ratio_y,
        "ps_magnification": magnification_point,
        "es_magnification": magnification_extended,
    }
    # except Exception as e:
    #     print(f"Error processing lens ID {lens_id}: {e}")
    #     return None

Overwriting /home/paras/projects/PDSPL/notebooks/worker.py


In [6]:
import multiprocessing
import pandas as pd
from tqdm import tqdm
from astropy.table import Table
import astropy.units as u

# --- Import the worker function from the file we just created ---
# from projects.PDSPL.notebooks.worker import extract_lens_properties
from worker import extract_lens_properties

In [7]:
# define a sky area
sky_area_source = Quantity(value=100, unit="deg2")  # sky area of the source population
sky_area_deflector = Quantity(value=1000, unit="deg2")  # sky area of the deflector population
sky_area_lens = Quantity(value=1000, unit="deg2")  # sky area of the lens population


# define limits in the intrinsic deflector and source population (in addition to the skypy config
# file)
kwargs_deflector_cut = {"band": "g", "band_max": 28, "z_min": 0.01, "z_max": 2.5}
kwargs_source_cut = {"band": "g", "band_max": 28, "z_min": 0.1, "z_max": 5.0}

# Initiate deflector population class.
# lens_galaxies = deflectors.AllLensGalaxies(
#     red_galaxy_list=galaxy_simulation_pipeline.red_galaxies,
#     blue_galaxy_list=galaxy_simulation_pipeline.blue_galaxies,
#     kwargs_cut=kwargs_deflector_cut,
#     kwargs_mass2light=None,
#     cosmo=cosmo_true,
#     sky_area=sky_area_deflector,
# )
lens_galaxies = deflectors.EllipticalLensGalaxies(
    galaxy_list=red_galaxy_catalog,
    kwargs_cut=kwargs_deflector_cut,
    kwargs_mass2light=None,
    cosmo=cosmo_true,
    sky_area=sky_area_deflector,
    gamma_pl={"mean": 2.078, "std_dev": 0.16}, # Ref: Auger et al. 2010 [https://ui.adsabs.harvard.edu/abs/2010ApJ...724..511A/abstract], For SLACS lenses
)

# Initiate source population class.
source_galaxies = sources.Galaxies(
    galaxy_list=blue_galaxy_catalog,
    kwargs_cut=kwargs_source_cut,
    cosmo=cosmo_true,
    sky_area=sky_area_source,
    catalog_type="skypy",
)

# make galaxy-galaxy population class using LensPop
gg_lens_pop = LensPop(
    deflector_population=lens_galaxies,
    source_population=source_galaxies,
    cosmo=cosmo_true,
    sky_area=sky_area_lens,
    los_pop=LOSPop(los_bool=False),  # no line-of-sight population, can be separately included in the forecast as 1% uncertainty on beta_E
)

# make lensplot class for extracting rgb images
gg_plot = LensingPlots(gg_lens_pop, num_pix=100, coadd_years=10)

print(f"LensPop initialized. Potential deflectors: {gg_lens_pop.deflector_number} ; Potential sources: {gg_lens_pop.source_number}");


LensPop initialized. Potential deflectors: 25253059 ; Potential sources: 474717380


In [8]:
main_progress_bar = tqdm(np.arange(50, 70, 1), desc="Overall Progress", position=0)

for i in main_progress_bar:

    kwargs_lens_cut = {"min_image_separation": 1, "max_image_separation": 8,
                    "second_brightest_image_cut": {"i": 26,},
                    }

    # kwargs_lens_cut = {}
    selected_lenses = gg_lens_pop.draw_population(
        kwargs_lens_cuts=kwargs_lens_cut,
    )

    # Print the number of selected lenses
    print(f"       Number of selected lenses: {len(selected_lenses)}")


    # 2. RUN IN PARALLEL
    num_processes = multiprocessing.cpu_count()-5
    print(f"       Starting parallel processing with {num_processes} cores...")
    multiprocessing.set_start_method("spawn", force=True)

    with multiprocessing.Pool(processes=num_processes) as pool:
        # Use enumerate() to pass both the ID and the lens object to the worker
        results_list = list(tqdm(
            pool.imap(extract_lens_properties, enumerate(selected_lenses)),
            total=len(selected_lenses),
            desc="Calculating lens properties"
        ))

    # 3. PROCESS RESULTS
    results_list = [res for res in results_list if res is not None]

    if results_list:
        df = pd.DataFrame(results_list)
        final_table = Table.from_pandas(df)
        
        # Save the table to a FITS file
        output_path = f"{PROJECT_DIR}/data/FULL_LSST/GGL_{sky_area_lens.value}_SQDEG_NON_SIS_{i}.fits"
        final_table.write(output_path, format="fits", overwrite=True)

        print(f"       Results saved to {output_path}")
        
        print("\n       Processing complete. First 5 rows of the generated table:")
        print(final_table[:5])
    else:
        print("       No results were generated.")

Overall Progress:   5%|▌         | 1/20 [56:23<17:51:26, 3383.50s/it]

       Results saved to ~/projects/PDSPL/data/FULL_LSST/GGL_1000.0_SQDEG_NON_SIS_50.fits

       Processing complete. First 5 rows of the generated table:
lens_id         z_D         ...  es_magnification 
------- ------------------- ... ------------------
      0 0.33764914484662156 ... 2.5960878822536473
      1 0.34826571423941144 ... 1.7578554601705298
      2 0.10593794010741038 ... 12.129084281305616
      3  1.7631143825059603 ... 1.8894450014002624
      4  0.5499245697508854 ...  4.807546970783347


Drawing lens population: 100%|██████████| 25253059/25253059 [45:20<00:00, 9282.38it/s]


       Number of selected lenses: 29039
       Starting parallel processing with 15 cores...


/home/paras/sharmapyenv/lib64/python3.12/site-packages/lenstronomy/LensModel/Solver/epl_shear_solver.py:245: RuntimeWarning: invalid value encountered in multiply
  + stuff[1] * 1j
/home/paras/sharmapyenv/lib64/python3.12/site-packages/lenstronomy/LensModel/Solver/epl_shear_solver.py:245: RuntimeWarning: invalid value encountered in multiply
  + stuff[1] * 1j
/home/paras/sharmapyenv/lib64/python3.12/site-packages/lenstronomy/LensModel/Solver/epl_shear_solver.py:245: RuntimeWarning: invalid value encountered in multiply
  + stuff[1] * 1j
/home/paras/sharmapyenv/lib64/python3.12/site-packages/lenstronomy/LensModel/Solver/epl_shear_solver.py:245: RuntimeWarning: invalid value encountered in multiply
  + stuff[1] * 1j
/home/paras/sharmapyenv/lib64/python3.12/site-packages/lenstronomy/LensModel/Solver/epl_shear_solver.py:245: RuntimeWarning: invalid value encountered in multiply
  + stuff[1] * 1j
/home/paras/sharmapyenv/lib64/python3.12/site-packages/lenstronomy/LensModel/Solver/epl_shear_s

...

       Results saved to ~/projects/PDSPL/data/FULL_LSST/GGL_1000.0_SQDEG_NON_SIS_68.fits

       Processing complete. First 5 rows of the generated table:
lens_id        z_D         ...  es_magnification
------- ------------------ ... -----------------
      0 1.0251252047237405 ... 35.26860310590561
      1 1.0622898287297264 ... 3.362771774051447
      2 0.5440447966300249 ... 2.640638137007374
      3 1.8488094686636058 ... 4.517389189752069
      4 0.6595707367894306 ... 38.44747050594604


Drawing lens population: 100%|██████████| 25253059/25253059 [41:31<00:00, 10137.06it/s]

       Number of selected lenses: 29010
       Starting parallel processing with 15 cores...



/home/paras/sharmapyenv/lib64/python3.12/site-packages/lenstronomy/LensModel/Solver/epl_shear_solver.py:245: RuntimeWarning: invalid value encountered in multiply
  + stuff[1] * 1j
/home/paras/sharmapyenv/lib64/python3.12/site-packages/lenstronomy/LensModel/Solver/epl_shear_solver.py:245: RuntimeWarning: invalid value encountered in multiply
  + stuff[1] * 1j
/home/paras/sharmapyenv/lib64/python3.12/site-packages/lenstronomy/LensModel/Solver/epl_shear_solver.py:245: RuntimeWarning: invalid value encountered in multiply
  + stuff[1] * 1j
/home/paras/sharmapyenv/lib64/python3.12/site-packages/lenstronomy/LensModel/Solver/epl_shear_solver.py:245: RuntimeWarning: invalid value encountered in multiply
  + stuff[1] * 1j
/home/paras/sharmapyenv/lib64/python3.12/site-packages/lenstronomy/LensModel/Solver/epl_shear_solver.py:245: RuntimeWarning: invalid value encountered in multiply
  + stuff[1] * 1j
/home/paras/sharmapyenv/lib64/python3.12/site-packages/lenstronomy/LensModel/Solver/epl_shear_

       Results saved to ~/projects/PDSPL/data/FULL_LSST/GGL_1000.0_SQDEG_NON_SIS_69.fits

       Processing complete. First 5 rows of the generated table:
lens_id         z_D         ...  es_magnification 
------- ------------------- ... ------------------
      0  0.2521190712735504 ...  5.556471908673621
      1 0.12996650298901385 ...  4.400454751256371
      2  0.8091984229767687 ...  6.392472862644283
      3  0.9445835108110998 ...  2.346130180151695
      4  0.9560589433143336 ... 1.8959245584224973


## Make Single Table for 20,000 sq.deg

In [ ]:
# filenames_1000deg_catalogs = [f"{PROJECT_DIR}/data/FULL_LSST/GGL_{1000.0}_SQDEG_NON_SIS_{i}.fits" for i in range(TIMES_TO_RUN)]
# fits_tables = [Table.read(filename, format="fits") for filename in filenames_1000deg_catalogs]
# combined_table = vstack(fits_tables)
# combined_table['lens_id'] = np.arange(len(combined_table))
# # save the combined table
# combined_table.write(f"{PROJECT_DIR}/data/FULL_LSST/GGL_{20000.0}_SQDEG_NON_SIS_0.fits", format="fits", overwrite=True)

In [9]:
# filenames_1000deg_catalogs = [f"{PROJECT_DIR}/data/FULL_LSST/GGL_{1000.0}_SQDEG_NON_SIS_{i}.fits" for i in range(50, 70, 1)]
# fits_tables = [Table.read(filename, format="fits") for filename in filenames_1000deg_catalogs]
# combined_table = vstack(fits_tables)
# combined_table['lens_id'] = np.arange(len(combined_table))
# # save the combined table
# combined_table.write(f"{PROJECT_DIR}/data/FULL_LSST/GGL_{20000.0}_SQDEG_NON_SIS_1.fits", format="fits", overwrite=True)

## Make Single Table for 50,000 sq.deg

In [ ]:
# filenames_1000deg_catalogs = [f"{PROJECT_DIR}/data/FULL_LSST/GGL_{1000.0}_SQDEG_NON_SIS_{i}.fits" for i in range(50)]
# fits_tables = [Table.read(filename, format="fits") for filename in filenames_1000deg_catalogs]
# combined_table = vstack(fits_tables)
# combined_table['lens_id'] = np.arange(len(combined_table))
# # save the combined table
# combined_table.write(f"{PROJECT_DIR}/data/FULL_LSST/GGL_{50000.0}_SQDEG_NON_SIS.fits", format="fits", overwrite=True)